# Computer Vision, Lab 3: Segmentation

Today we'll experiment with methods to segment a scene. Since we've been working with a sample ground robot in an indoor environment, we'll take as an example the task of segmenting unoccupied ground plane space from obstacles.

If we can successfully determine which pixels in the image seen by our robot are on the flat floor and which are likely obstacles, we can combine that information with the ground plane homography-based rectification method we developed in the last lab to obtain a map of the space around the robot.

The same approach could be used by an autonomous vehicle to determine where, in the image from its front-facing camera, the road is and where possible obstacles are.

## Background: Color-based segmentation

In our indoor example, we have fairly consistent lighting and a consistently-colored floor, so a color based approach may work.

The task here is, given an input pixel at location $(x,y)$ with RGB color $(r,g,b)$, classify the pixel as "floor" or "not floor."

This is a classic machine learning problem. We have an input feature vector $(r, g, b)$ (some methods would also utilize $x$ and $y$ as well), and a desired output of 1
for floor and 0 for non-floor.

We probably want to transform the color space from RGB to HSV, since the RGB vector mixes color information with intensity information in the same measurements:

<img src="img/lab03-1.jpg" width="300"/>

whereas the HSV color space separates color from saturation ("color purity") and intensity:

<img src="img/lab03-2.jpg" width="300"/>

(Images are from Wikimedia Commons via the OpenCV documentation.)


Although we could use a fancy classifier like logistic regression or a SVM, this problem is easily solved with a generative model. We apply the principle of maximum a posteriori classification and Bayes' rule:

$$
f(h, s, v) = 
    \begin{cases}
        1 & p(\text{floor} \mid h, s, v) > 0.5 \\
        0 & \text{otherwise}
    \end{cases}
$$

$$
p(\text{floor} \mid h, s, v) = \frac{p(h, s, v \mid \text{floor}) p(\text{floor})}{p(h, s, v)}
$$

and since $p(h, s, v)$ is not known, we eliminate it:

$$
f(h, s, v) =
    \begin{cases}
        1 & p(h, s, v \mid \text{floor}) p(\text{floor}) > p(h, s, v | \text{not floor}) p(\text{not floor}) \\
        0 & \text{otherwise}
    \end{cases}
$$

The entity $p(\text{floor)$ is easy to estimate as the proportion of pixels in a sample of images that are floor pixels.

What about $p(h, s, v | floor)$?

The simplest approach here is just a lookup table. Since we're conditioning on floor, we just need to sample a bunch of floor pixels and record the frequency of each value of $h$, $s$, and $v$.

The problem with that approach is that we'd need something like $10 * 255 * 255 * 255$ pixels (168M!) to get a decent estimate of the value in every bin of this frequency table.

What we do instead is collapse some values of $h$, $s$, and $v$ into bins. For example, we might drop the 4 least significant bits of each of these values, then we'd end up needing only about $10 * 16 * 16 * 16$ (just 41K) samples to get a decent estimate.

Note that some methods would further drop the V element, which represents intensity, on the assumption that the global amount of lighting doesn't matter.

In fact, as the sun moves across the sky, the color composition of the light changes, and artificial light is quite different in color than sunlight. Still, it's a start.

So we want to collapse the $(h, s, v)$ vector into two features, $(h>>4, s>>4)$. We'll end up with a lookup table containing just $16*16 = 256$ elements.

## Create and save an image-plane-to-ground-plane homography to a YML file

Let's use a simple version of the program from lab 2 that calls `getPerspectiveTransform()`
to get a ground plane to image plane homography and save it to a YML file.

In [ ]:
#include <iostream>
#include <opencv2/opencv.hpp>

using namespace cv;
using namespace std;

#define VIDEO_FILE "robot.mp4"
#define HOMOGRAPHY_FILE "robot-homography.yml"

Mat matPauseScreen, matResult, matFinal;
Point point;
vector<Point> pts;
int var = 0;
int drag = 0;

// Create mouse handler function
void mouseHandler(int event, int x, int y, int, void*)
{
    if (var >= 4) return;
    if (event == EVENT_LBUTTONDOWN) // Left button down
    {
        drag = 1;
        matResult = matFinal.clone();
        point = Point(x, y);
        if (var >= 1) 
        {
            line(matResult, pts[var - 1], point, Scalar(0, 255, 0, 255), 2);
        }
        circle(matResult, point, 2, Scalar(0, 255, 0), -1, 8, 0);
        imshow("Source", matResult);
    }
    if (event == EVENT_LBUTTONUP && drag) // When Press mouse left up
    {
        drag = 0; var++;
        pts.push_back(point);
        matFinal = matResult.clone();
        if (var >= 4)
        {
            line(matFinal, pts[0], pts[3], Scalar(0, 255, 0, 255), 2);
            fillPoly(matFinal, pts, Scalar(0, 120, 0, 20), 8, 0);

            setMouseCallback("Source", NULL, NULL);
        }
        imshow("Source", matFinal);
    }
    if (drag)
    {
        matResult = matFinal.clone();
        point = Point(x, y);
        if (var >= 1) 
            line(matResult, pts[var - 1], point, Scalar(0, 255, 0, 255), 2);
        circle(matResult, point, 2, Scalar(0, 255, 0), -1, 8, 0);
        imshow("Source", matResult);
    }
}

int main()
{
    Mat matFrameCapture;
    Mat matFrameDisplay;
    int key = -1;

    VideoCapture videoCapture(VIDEO_FILE);
    if (!videoCapture.isOpened()) {
        cerr << "ERROR! Unable to open input video file " << VIDEO_FILE << endl;
        return -1;
    }

    while (key < 0)        // play video until press any key
    {
        // Get the next frame
        videoCapture.read(matFrameCapture);
        if (matFrameCapture.empty()) {
            // End of video file
            break;
        }
        float ratio = 640.0 / matFrameCapture.cols;
        resize(matFrameCapture, matFrameDisplay, cv::Size(), ratio, ratio, INTER_LINEAR);

        imshow(VIDEO_FILE, matFrameDisplay);
        key = waitKey(30);

        if (key >= 0)
        {
            destroyWindow(VIDEO_FILE);
            matPauseScreen = matFrameCapture;
            matFinal = matPauseScreen.clone();

            namedWindow("Source", WINDOW_AUTOSIZE);
            setMouseCallback("Source", mouseHandler, NULL);
            imshow("Source", matPauseScreen);
            waitKey(0);
            destroyWindow("Source");

            Point2f src[4];
            for (int i = 0; i < 4; i++)
            {
                src[i].x = pts[i].x * 1.0;
                src[i].y = pts[i].y * 1.0;
            }
            Point2f reals[4];
            reals[0] = Point2f(800.0, 800.0);
            reals[1] = Point2f(1000.0, 800.0);
            reals[2] = Point2f(1000.0, 1000.0);
            reals[3] = Point2f(800.0, 1000.0);

            Mat homography_matrix = getPerspectiveTransform(src, reals);
            std::cout << "Estimated Homography Matrix is:" << std::endl;
            std::cout << homography_matrix << std::endl;

            // perspective transform operation using transform matrix
            cv::warpPerspective(matPauseScreen, matResult, homography_matrix, matPauseScreen.size(), cv::INTER_LINEAR);
            imshow("Source", matPauseScreen);
            imshow("Result", matResult);

            waitKey(0);
        }
    }

    return 0;
}

### Python

In [ ]:
import cv2
import numpy as np

VIDEO_FILE = "robot.mp4"
HOMOGRAPHY_FILE = "robot-homography.yml"

matResult = None
matFinal = None
matPauseScreen = None

point = (-1, -1)
pts = []
var = 0 
drag = 0

# Mouse handler function has 5 parameters input (no matter what)
def mouseHandler(event, x, y, flags, param):
    global point, pts, var, drag, matFinal, matResult

    if (var >= 4):                           # if homography points are more than 4 points, do nothing
        return
    if (event == cv2.EVENT_LBUTTONDOWN):     # When Press mouse left down
        drag = 1                             # Set it that the mouse is in pressing down mode
        matResult = matFinal.copy()          # copy final image to draw image
        point = (x, y)                       # memorize current mouse position to point var
        if (var >= 1):                       # if the point has been added more than 1 points, draw a line
            cv2.line(matResult, pts[var - 1], point, (0, 255, 0, 255), 2)    # draw a green line with thickness 2
        cv2.circle(matResult, point, 2, (0, 255, 0), -1, 8, 0)             # draw a current green point
        cv2.imshow("Source", matResult)      # show the current drawing
    if (event == cv2.EVENT_LBUTTONUP and drag):  # When Press mouse left up
        drag = 0                             # no more mouse drag
        pts.append(point)                    # add the current point to pts
        var += 1                             # increase point number
        matFinal = matResult.copy()          # copy the current drawing image to final image
        if (var >= 4):                                                      # if the homograpy points are done
            cv2.line(matFinal, pts[0], pts[3], (0, 255, 0, 255), 2)   # draw the last line
            cv2.fillConvexPoly(matFinal, np.array(pts, 'int32'), (0, 120, 0, 20))        # draw polygon from points
        cv2.imshow("Source", matFinal);
    if (drag):                                    # if the mouse is dragging
        matResult = matFinal.copy()               # copy final images to draw image
        point = (x, y)                            # memorize current mouse position to point var
        if (var >= 1):                            # if the point has been added more than 1 points, draw a line
            cv2.line(matResult, pts[var - 1], point, (0, 255, 0, 255), 2)    # draw a green line with thickness 2
        cv2.circle(matResult, point, 2, (0, 255, 0), -1, 8, 0)         # draw a current green point
        cv2.imshow("Source", matResult)           # show the current drawing

def main():
    global matFinal, matResult, matPauseScreen
    key = -1;

    videoCapture = cv2.VideoCapture(VIDEO_FILE)
    if not videoCapture.isOpened():
        print("ERROR! Unable to open input video file ", VIDEO_FILE)
        return -1

    width  = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`

    # Capture loop 
    while (key < 0):
        # Get the next frame
        _, matFrameCapture = videoCapture.read()
        if matFrameCapture is None:
            # End of video file
            break

        ratio = 640.0 / width
        dim = (int(width * ratio), int(height * ratio))
        matFrameDisplay = cv2.resize(matFrameDisplay, dim)

        cv2.imshow(VIDEO_FILE, matFrameDisplay)
        key = cv2.waitKey(30)

        if (key >= 0):
            cv2.destroyWindow(VIDEO_FILE)
            matPauseScreen = matFrameCapture
            matFinal = matPauseScreen.copy()
            cv2.namedWindow("Source", cv2.WINDOW_AUTOSIZE)
            cv2.setMouseCallback("Source", mouseHandler)
            cv2.imshow("Source", matPauseScreen)
            cv2.waitKey(0)
            cv2.destroyWindow("Source")

            if (len(pts) < 4):
                return

            src = np.array(pts).astype(np.float32)
            reals = np.array([(800, 800),
                                (1000, 800),
                                (1000, 1000),
                                (800, 1000)], np.float32)

            homography_matrix = cv2.getPerspectiveTransform(src, reals);
            print("Estimated Homography Matrix is:")
            print(homography_matrix)

            h, w, ch = matPauseScreen.shape
            matResult = cv2.warpPerspective(matPauseScreen, homography_matrix, (w, h), cv2.INTER_LINEAR)
            matPauseScreen = cv2.resize(matPauseScreen, dim)
            cv2.imshow("Source", matPauseScreen)
            matResult = cv2.resize(matResult, dim)
            cv2.imshow("Result", matResult)

            cv2.waitKey(0)

main()

## Create a class for reading/writing homography data

Let's make a structure for our homography data. Objects
of class <code>HomographyData</code> will keep the information about a homography. It contains:

- cPoints: number of points for setting homography
- aPoints: points information
- matH: Homography matrix
- widthOut: image width of the output image
- heightOut: image height of the output image

We need two methods: reading and writing homographies.

At this step, you should create a new cpp file for create the special functions and structure. Please create 2 files: <code>HomographyData.h</code> and <code>HomographyData.cpp</code>

For Visual Studio users: <code>HomographyData.h</code> should be created in your
"Header Files" section, and <code>HomographyData.cpp</code> should be created in
your "Source Files" section.


### C++ header

Place the following in `HomographyData.h`:

In [ ]:
#include <opencv2/opencv.hpp>

class HomographyData
{
public:
    cv::Mat matH;
    int widthOut;
    int heightOut;
    int cPoints;
    cv::Point2f aPoints[4];

    HomographyData();
    HomographyData(std::string homography_file);

    bool read(std::string homography_file);
    bool write(std::string homography_file);
};

### C++ source file

Then provide the implementation in `HomographyData.cpp`:

In [ ]:
#include "homography.h"

Homography::Homography(std::string homography_file)
{
    read(homography_file);
}

HomographyData::HomographyData()
{
    cPoints = 0;
}

bool HomographyData::read(std::string homography_file)
{
    cv::FileStorage fileStorage(homography_file, cv::FileStorage::Mode::READ);
    if (!fileStorage.isOpened()) {
        return false;
    }
    cv::FileNode points = fileStorage["aPoints"];
    cv::FileNodeIterator it = points.begin(), it_end = points.end();
    cPoints = 0;
    for (int i = 0; it != it_end; it++, i++) {
        (*it) >> aPoints[i];
        cPoints++;
    }
    fileStorage["matH"] >> matH;
    fileStorage["widthOut"] >> widthOut;
    fileStorage["heightOut"] >> heightOut;
    fileStorage.release();
    return true;
}

bool HomographyData::write(std::string homography_file)
{
    cv::FileStorage fileStorage(homography_file, cv::FileStorage::Mode::WRITE);
    if (!fileStorage.isOpened()) {
        return false;
    }

    fileStorage << "aPoints" << "[";
    for (int i = 0; i < 4; i++)
    {
        fileStorage << aPoints[i];
    }
    fileStorage << "]";
    fileStorage << "matH" << matH;
    fileStorage << "widthOut" << widthOut;
    fileStorage << "heightOut" << heightOut;
    fileStorage.release();
    return true;
}

### Python

In [ ]:
import cv2
import numpy as np
from typing import List #use it for :List[...]

class Homograpy:
    matH = np.zeros((3, 3))
    widthOut : int
    heightOut : int
    cPoints : int
    aPoints:list = []

    def __init__(self, homography_file = None):
        self.cPoints = 0
        if homography_file is not None:
            self.read(homography_file)

    def read(self, homography_file):
        fileStorage = cv2.FileStorage(homography_file, cv2.FILE_STORAGE_READ)
        if not fileStorage.isOpened():
            return False

        self.cPoints = 0
        for i in range(points.size()):
            points = fileStorage.getNode("aPoints" + str(i))
            self.aPoints.append(points.mat())
            self.cPoints += 1
        self.matH = fileStorage.getNode("matH").mat()
        self.widthOut = int(fileStorage.getNode("widthOut").real())
        self.heightOut = int(fileStorage.getNode("heightOut").real())
        fileStorage.release()
        return True

    def write(self, homography_file):
        fileStorage = cv2.FileStorage(homography_file, cv2.FILE_STORAGE_WRITE)
        if not fileStorage.isOpened():
            return False

        for i in range(4):
            fileStorage.write("aPoints" + str(i), self.aPoints[i])

        fileStorage.write("matH", self.matH)
        fileStorage.write("widthOut", self.widthOut)
        fileStorage.write("heightOut", self.heightOut)
        fileStorage.release()
        return True

### Using it in C++

In [ ]:
#include "HomographyData.h"

...

// Write H to file

HomographyData homographyData;
for (int i = 0; i < 4; i++)
{
    homographyData.aPoints[i] = src[i];
    homographyData.cPoints++;
}
homographyData.matH = homography_matrix;
homographyData.widthOut = matPauseScreen.cols;
homographyData.heightOut = matPauseScreen.rows;
if (!homographyData.write(HOMOGRAPHY_FILE)) {
    cerr << "ERROR! Unable to write homography data file " << HOMOGRAPHY_FILE << endl;
    return -1;
}

...

// Read H from file

HomographyData homographyData(HOMOGRAPHY_FILE);
if (!homographyData.cPoints == 0) {
    cerr << "ERROR! Unable to read homography data file " << HOMOGRAPHY_FILE << endl;
    return -1;
}

### Using it in Python

In [ ]:
# Write H to file

homographyData = Homography()
homographyData.cPoints = 0
for i in range(4):
    homographyData.aPoints.append(src[i])
    homographyData.cPoints += 1
homographyData.matH = homography_matrix
homographyData.widthOut = width
homographyData.heightOut = height
homographyData.write(HOMOGRAPHY_FILE)

...

# Read H from file

homographyData = Homography(HOMOGRAPHY_FILE)
if homographyData.cPoints == 0:
    print("ERROR! Unable to read homography data file ", HOMOGRAPHY_FILE)
    return -1

### Results

The output file `robot-homography.yml` might contain the following:

    %YAML:1.0
    ---
    aPoints0: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 860., 49. ]
    aPoints1: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 1386., 52. ]
    aPoints2: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 1620., 375. ]
    aPoints3: !!opencv-matrix
       rows: 2
       cols: 1
       dt: d
       data: [ 784., 367. ]
    matH: !!opencv-matrix
       rows: 3
       cols: 3
       dt: d
       data: [ -1.1391186748382573e-03, -2.2794919900117339e-03,
           1.0627648167793186e-16, 1.6604445559011895e-05,
           -3.2917310423991194e-03, -3.1863931027743840e-02,
           -1.6241141452077307e-09, -1.8250980274059071e-06,
           -9.0126408686546570e-04 ]
    widthOut: 2419
    heightOut: 1250

## Use a pre-computed homography in a new program

### C++

In [ ]:
#include <opencv2/opencv.hpp>
#include <iostream>
#include "homography.h"

using namespace cv;
using namespace std;

#define VIDEO_FILE "robot.mp4"
#define HOMOGRAPHY_FILE "robot-homography.yml"

void displayFrame(cv::Mat& matFrameDisplay, int iFrame, int cFrames, tsHomographyData* pHomographyData) {
    for (int i = 0; i < pHomographyData->cPoints; i++) {
        cv::circle(matFrameDisplay, pHomographyData->aPoints[i], 10, cv::Scalar(255, 0, 0), 2, cv::LINE_8, 0);
    }
    imshow(VIDEO_FILE, matFrameDisplay);
    stringstream ss;
    ss << "Frame " << iFrame << "/" << cFrames;
    ss << ": hit <space> for next frame or 'q' to quit";
    //cv::displayOverlay(VIDEO_FILE, ss.str(), 0);  // for linux + qt
    putText(matFrameDisplay, ss.str(), Point(10, 30), FONT_HERSHEY_SIMPLEX, 1.0, Scalar(0, 0, 255), 3);
}

int main()
{
    cv::Mat matFrameCapture;
    cv::Mat matFrameDisplay;
    int cFrames;
    tsHomographyData homographyData;

    cv::VideoCapture videoCapture(VIDEO_FILE);
    if (!videoCapture.isOpened()) {
        cerr << "ERROR! Unable to open input video file " << VIDEO_FILE << endl;
        return -1;
    }
    cFrames = (int)videoCapture.get(cv::CAP_PROP_FRAME_COUNT);

    // Create a named window that will be used later to display each frame
    cv::namedWindow(VIDEO_FILE, (unsigned int)cv::WINDOW_NORMAL | cv::WINDOW_KEEPRATIO | cv::WINDOW_GUI_EXPANDED);

    // Read homography from file
    if (!readHomography(HOMOGRAPHY_FILE, &homographyData)) {
        cerr << "ERROR! Unable to read homography data file " << HOMOGRAPHY_FILE << endl;
        return -1;
    }

    int iFrame = 0;
    while (true) {

        // Block for next frame

        videoCapture.read(matFrameCapture);
        if (matFrameCapture.empty()) {
            // End of video file
            break;
        }

        displayFrame(matFrameCapture, iFrame, cFrames, &homographyData);

        int iKey;
        do {
            iKey = cv::waitKey(10);
            if (getWindowProperty(VIDEO_FILE, cv::WND_PROP_VISIBLE) == 0) {
                return 0;
            }
            if (iKey == (int)'q' || iKey == (int)'Q') {
                return 0;
            }
        } while (iKey != (int)' ');
        iFrame++;
    }

    return 0;
}

### Python

In [ ]:
import cv2
import numpy as np
from homography import tsHomographyData, readHomography, writeHomography

VIDEO_FILE = "robot.mp4"
HOMOGRAPHY_FILE = "robot-homography.yml"

def displayFrame(matFrameDisplay, iFrame, cFrames, pHomographyData):
    for i in range(pHomographyData.cPoints):
        cv2.circle(matFrameDisplay, pHomographyData.aPoints[i], 10, (255, 0, 0), 2, cv.LINE_8, 0)

    cv2.imshow(VIDEO_FILE, matFrameDisplay)
    ss = "Frame " + str(iFrame) + "/" + str(cFrames)
    ss += ": hit <space> for next frame or 'q' to quit";
    # cv2.displayOverlay(VIDEO_FILE, ss, 0);  # for linux + qt
    cv2.putText(matFrameDisplay, ss, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3);

def main():
    global matFinal, matResult, matPauseScreen
    key = -1;

    videoCapture = cv2.VideoCapture(VIDEO_FILE)
    if not videoCapture.isOpened():
        print("ERROR! Unable to open input video file ", VIDEO_FILE)
        return -1

    width  = videoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    height = videoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`

    cFrames = videoCapture.get(cv2.CAP_PROP_FRAME_COUNT)

    cv2.namedWindow(VIDEO_FILE, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_EXPANDED)

    homographyData = readHomography(HOMOGRAPHY_FILE)
    if homographyData is None:
        print("ERROR! Unable to read homography data file ", HOMOGRAPHY_FILE)
        return -1

    iFrame = 0
    # Capture loop 
    while (key < 0):
        # Get the next frame
        _, matFrameCapture = videoCapture.read()
        if matFrameCapture is None:
            # End of video file
            break

        matFrameCapture = videoCapture.read(matFrameCapture)
        if matFrameCapture is None:
            # End of video file
            break

        displayFrame(matFrameCapture, iFrame, cFrames, homographyData)

        iKey = -1
        while iKey != ord(' '):
            iKey = cv2.waitKey(10)
            if (getWindowProperty(VIDEO_FILE, cv2.WND_PROP_VISIBLE) == 0):
                return 0
            if (iKey == ord('q') or iKey == ord('Q')):
                return 0

        iFrame += 1

        return

main()

## Quick version of segmentation

As a simple experiment, find a pixel in the image that is clearly on the floor, convert the input image to the HSV colorspace using code

    cvtColor(matFrameBGR, matFrameHSV, COLOR_BGR2HSV);

If you display HSV as an image you might get something like this:

<img src="img/lab03-4.png" width="600"/>

Output the HSV values at your preferred location. Create a 2D array <code>double aProbFloorHS[16][16]</code> and set the selected bin to a probability of 1.0 and all others to 0.
Show, in a second window, a mask for the pixels selected by $p(h, s, v \mid \text{floor}) > 0.5$.

## Obtain HS probabilities using a machine learning model

Next, take the first frame from the video, load it in <link>[gimp](https://www.gimp.org/downloads/)</link> or other image editing software, and make a mask for the floor pixels.

To extract frames from the video at a frame rate of one frame per second of the video, try at the command line

    $ ffmpeg -i robot.mp4 -r 1 -f image2 frame-%03d.png

Or you can save the images in your OpenCV program:

    stringstream ss;
    ss << fixed << setprecision(3) << setfill('0');     // set fix digit prefix length to 3 and the blank digit is filled by 0
    ss << "frame-" << setw(3) << iFrame << ".png";      // set iFrame length to 3 from command above
    imwrite(ss.str(), matFrameCapture);

To make a mask in gimp, add a 50% transparent layer, color pixels you want to select, and delete the original image layer with: 

(right click $\rightarrow$ Layer $\rightarrow$ Stack $\rightarrow$ Select Next Layer), and then

(right click $\rightarrow$ Layer $\rightarrow$ Delete Layer).

<img src="img/lab03-5.png" width="300"/>

<img src="img/lab03-6.png" width="600"/>

<img src="img/lab03-7.png" width="600"/>

You can copy the resulting layer then create a new image from the clipboard and export as a black and white PNG.

<img src="img/lab03-8.png" width="600"/>

Add code to your program to read the mask, apply it to the first frame of the video, and calculate the entries in the <code>aProbFloorHS</code> array. You'll also need a <code>aProbNonFloorHS</code> array and total pixel counts.

Note: The code is just an example, it might have error or incorrect.

### C++

In [ ]:
double aProbFloorHS[16][16];
double aProbNonFloorHS[16][16];
void readProbfloor(string mask_file)
{
    Mat mask = imread(mask_file);
    cvtColor(mask, mask, COLOR_BGR2GRAY);
    int scaleH = matFrameHSV.rows / 16;
    int scaleW = matFrameHSV.cols / 16;
    double area = scaleH * scaleW * 1.0;
    for (int ax = 0; ax < 16; ax++)
    {
        for (int ay = 0; ay < 16; ay++)
        {
            int floor = 0;
            for (int x = 0; x < scaleW; x++)
            {
                for (int y = 0; y < scaleH; y++)
                {
                    int pix = mask.at<int>(ay * scaleH + y, ax * scaleW + x);
                    if (pix > 128)
                        floor++;
                }
            }
            aProbFloorHS[ax][ay] = floor / area;
            aProbNonFloorHS[ax][ay] = 1 - aProbFloorHS[ax][ay];
        }
    }
}

### Python

In [ ]:
aProbFloorHS = np.zeros((16,16))
aProbNonFloorHS = np.zeros((16,16))

void readProbfloor(mask_file)
{
    mask = cv2.imread(mask_file)
    cv2.cvtColor(mask, mask, cv2.COLOR_BGR2GRAY)
    scaleH = matFrameHSV.rows // 16
    scaleW = matFrameHSV.cols // 16
    area = scaleH * scaleW * 1.0
    for ax in range(16):
        for ay in range(16):
            floor = 0
            for x in range(scaleW):
                for y in range(scaleH):
                    pix = mask[ay * scaleH + y, ax * scaleW + x]
                    if (pix > 128):
                        floor += 1
            aProbFloorHS[ax,ay] = floor / area;
            aProbNonFloorHS[ax,ay] = 1 - aProbFloorHS[ax,ay]

## Try to get a good probabilistic floor color model yourself

Finally, display the mask in a partially transparent color on top of the image as the video is rendered. How well does it work? You might want to add additional images to your training set. Consider a tool such as hasty.ai to mark up multiple images.

With a training set of 19 images, 64 bins for H, 16 bins for S, and 16 bins for V, I got a leave-one-out cross validated test accuracy of around 95%, with an F1 for the floor of 0.98 and an F1 for the obstacles of 0.51. See how well your model works and include this information in your report.


## Semantic Segmentation

We've seen that color-based segmentation in HSV space using a generative machine learning model can be effective but has limitations when the objects to be segmented from the background have color distributions similar to the background. The best accuracy we could get for our floor/obstacle model last week was around 95%, but hat was with occasional errors grouping in large enough regions that would preclude our robot from navigating safely in the indoor environment.

Semantic segmentation attempts to address this issue using a more sophisticated model to separate the scene into its constituent regions more effectively.

Typical semantic segmentation models typically use a lot of resources. For example, the state of the art models published for the MIT ADE20K dataset in their <link>[GitHub repository](https://github.com/CSAILVision/semantic-segmentation-pytorch)</link> are very accurate and run at 2-17 FPS on a NVIDIA Pascal Titan Xp GPU. They also run well on a GTX 1080TI. But on an i7 CPU, I found that they take 9-11 SECONDS PER FRAME, and on an NVIDIA Jetson Nano's GPU, they take 12-42 SEDONDS PER FRAME!

## Lighter semantic segmentation models

We would like to experiment with some semantic segmenation models that have a hope of running in real time on a small embedded system such as the Jetson Nano.

NVIDIA has published a very useful repository <link>[Jetson Inference](https://github.com/dusty-nv/jetson-inference)</link> that contains versions of two semantic segmentation models: SegNet and UNet.

I ran the SegNet model in this repository using FCN-ResNet18 trained on Pascal VOC with 320x320 input images. It takes a while to load the model into memory and so on, but inference time once all is ready is very fast: less than 70 ms.

So it's fast! Unfortunately, it doesn't work particularly well out of the box:

<img src="img/lab03-3.png" width="600"/>

But we shouldn't expect it to, considering that the Pascal VOC dataset "only" contains 20 classes plus the "background" (the black label), and others of which are "bottle" (the purple label) and "person" (the tan label).

That's on the NVIDIA Jetson Nano. The model itself was built with PyTorch, but it has been exported in ONNX (Open Neural Network Exchange) format, and the Jetson Nano executes it on TensorRT. You can download the <link>[ONNX model from AIT](https://www.cs.ait.ac.th/~mdailey/class/vision/fcn_resnet18.onnx)</link> (I just copied it from the excellent Jetson Inference repository).

If you'd like to understand the structure of the model represented in this ONNX file, download the <link>[Netron 4.3.4 AppImage for Linux](https://github.com/lutzroeder/netron)</link>, run the AppImage, and load the file. You'll see that it

1. Takes as input a 320x320 3-channel image
2. Performs 64 7x7 convolutions
3. Does batch normalization, ReLU, and MaxPool
4. Runs a residual block with the following
    1. 64 3x3 convolutions then BN then ReLU
    2. 64 3x3 convolutions then BN
5. ReLU
6. Residual block (same structure as above)
7. ReLU
8. Residual block with the following
    1. 128 3x3 convolutions then BN then ReLU
    2. 128 3x3 convolutions then BN
    3. Residual add using 128 1x1 convs to expand feature map
9. Etc. (several more residual blocks and downscaling by a factor of 32)
10. Final 21 1x1 convolution to obtain output 1x21x10x10 tensor

Since this model is relatively small and at least generates some output, let's see if we can get it running on our OpenCV video stream. A model that runs nice and fast on an Intel CPU and on a Jetson GPU, in C++/OpenCV as well as Python, would be perfect.

<img src="img/lab03-10.png" width="300"/>

### FCN ResNet 18

Let's see if we can get this ONNX model running in OpenCV.

First, let's initialize with the Pascal VOC classes and read an image:

### C++

In [ ]:
string aStringClasses[] = {
    "background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
};

cv::Vec3b aColorClasses[] = {
        { 0, 0, 0 }, { 255, 0, 0 }, { 0, 255, 0 }, { 0, 255, 120 }, { 0, 0, 255 }, { 255, 0, 255 }, { 70, 70, 70 },
        { 102, 102, 156 }, { 190, 153, 153 }, { 180, 165, 180 }, { 150, 100, 100 }, { 153, 153, 153 },
        { 250, 170, 30 }, { 220, 220, 0 }, { 107, 142, 35 }, { 192, 128, 128 }, { 70, 130, 180 }, { 220, 20, 60 },
        { 0, 0, 142 }, { 0, 0, 70 }, { 119, 11, 32 }
};

int nClasses = sizeof(aColorClasses) / 3;

// Read CNN definition

auto net = cv::dnn::readNetFromONNX(ONNX_NETWORK_DEFINITION);

// Read input image

cv::Mat matFrame = cv::imread(IMAGE_FILE);
if (matFrame.empty()) {
    cerr << "Cannot open image file " << IMAGE_FILE << endl;
    return -1;
}

### Python

In [ ]:
aStringClasses = [
    "background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

aColorClasses = [
        ( 0, 0, 0 ), ( 255, 0, 0 ), ( 0, 255, 0 ), ( 0, 255, 120 ), ( 0, 0, 255 ), ( 255, 0, 255 ), ( 70, 70, 70 ),
        ( 102, 102, 156 ), ( 190, 153, 153 ), ( 180, 165, 180 ), ( 150, 100, 100 ), ( 153, 153, 153 ),
        ( 250, 170, 30 ), ( 220, 220, 0 ), ( 107, 142, 35 ), ( 192, 128, 128 ), ( 70, 130, 180 ), ( 220, 20, 60 ),
        ( 0, 0, 142 ), ( 0, 0, 70 ), ( 119, 11, 32 )
]

nClasses = len(aColorClasses)

# Read CNN definition
net = cv2.dnn.readNetFromONNX(cv2.ONNX_NETWORK_DEFINITION)

# Read input image
matFrame = cv2.imread(IMAGE_FILE);
if (matFrame is None):
    print("Cannot open image file ", IMAGE_FILE)
    return -1

Once the input image is preprocessed to form a tensor suitable for input to our DNN model, we can just set the input layer of the network to point to the newly preprocessed image tensor, then we can do a forward pass through the network model:

### C++

In [ ]:
// Propagate the matInputTensor through the FCN model

net.setInput(matInputTensor);
cv::Mat matScore = net.forward();

### Python

In [ ]:
# Propagate the matInputTensor through the FCN model
net.setInput(matInputTensor)
matScore = net.forward()

OK but what about the preprocessing?

Here's the thing: most CNN models (even object detection and image segmentation models) are based on a classification model as the "backbone" of the model. In the case of FCN-ResNet-18, the backbone is of course ResNet-18.

Usually, training a model based on a classifier begins by loading weights trained for classification on ImageNet or another dataset then further training and/or "fine tuning" the model on a more specific dataset. We do this because we don't want to spend the week of GPU time it takes to get a model that analyzes image edges, puts them together into higher-level shapes, and gradually extracts a set of coarsely localized features that describe objects of interest.

When data scientists train a model on ImageNet, they almost always do a few common steps:

1. Scale the input image's R, G, and B intensities (normally in the range 0-255) to the range 0-1.
2. Scale the input image to the size needed for the classifier, or sample a patch from the input the size required by the classifier.
3. Subtract expected mean values for the R, G, and B channels. The magic values for ImageNet are 0.485 for R, 0.456 for G, and 0.406 for B.
4. Divide by expected standard deviations for the R, G, and B channels. The magic values for ImageNet are 0.229 for R, 0.224 for G, and 0.225 for B.

There is an OpenCV function <code>cv::dnn::blobFromImage()</code> that performs some of these steps but not all. Check the documentation and add the necessary code to prepare the image for presentation for the pre-trained network.

Next, you'll want to use the result of the semantic segmentation model to color the input image and display for examination by the user, and perhaps add some information about CPU time required for the inference:

### C++

In [ ]:
// Colorize the image and display

cv::Mat matColored;
colorizeSegmentation(matFrame, matScore, matColored, aColorClasses, aStringClasses, nClasses);

// Add timing information

std::vector<double> layersTimes;
double freq = cv::getTickFrequency() / 1000;
double t = net.getPerfProfile(layersTimes) / freq;
std::string label = cv::format("Inference time: %.2f ms", t);
cv::putText(matColored, label, cv::Point(10,30),
        cv::FONT_HERSHEY_SIMPLEX, 1.0, cv::Scalar(0, 255, 0));

// Display

cv::namedWindow(WINDOW_NAME, WINDOW_FLAGS);
cv::imshow(WINDOW_NAME, matColored);
cv::waitKey(0);

return 0;

### Python

In [ ]:
# Colorize the image and display

matColored = cv2.colorizeSegmentation(matFrame, matScore, aColorClasses, aStringClasses, nClasses)

# Add timing information
layersTimes = 0
freq = cv2.getTickFrequency() / 1000;
t = net.getPerfProfile(layersTimes) / freq;
label = "Inference time: " + str(t) + " ms"
cv2.putText(matColored, label, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0))

# Display
cv2.namedWindow(WINDOW_NAME, WINDOW_FLAGS)
cv2.imshow(WINDOW_NAME, matColored)
cv2.waitKey(0)

You'll have to figure out colorizeSegmentation for yourself. See if you can get a result similar to the image above. It won't be exactly the same, as the jetson inference code scales the input image slightly differently from <code>blobFromImage()</code>. I got the following:

<img src="img/lab03-11.png" width="600"/>

### Fine tuning on our own dataset

Next, we will want to fine-tune the FCN-ResNet-18 model to our own floor/obstacle dataset. We will load the existing weights, throw away the 21-class output layer of the existing model, and replace it with our own two-class output layer. We'll keep the weights for all but this last layer, then start training on our dataset.

We saw that we could run a pre-trained FCN-ResNet-18 on our robot floor/obstacle video sequence using a model exported to ONNX format and executed using OpenCV's DNN module.

However, the stock model, which was trained on VOC 2012's 21 classes, was unable to give a recognizable segmentation on our data set.

Therefore, we learn how to train a semantic segmentation model on a benchmark dataset (VOC 2012) then fine tune that model to our dataset, with the hope that the result will be effective.

Also, since we will be training on a medium-sized dataset (VOC), we should set up our machine learning model development environment to use a powerful GPU server rather than our poor little laptops.

### GPU server setup

This lab should work fine on the AIT DS&AI JupyterHub
server <link>[login here](https://puffer.cs.ait.ac.th/hub/login?next=)</link>

You might instead want to create a custom environment according to
<link>[RTML GPU setup](https://github.com/dsai-asia/RTML/blob/main/Labs/01-Setup/01-Setup.ipynb)</link>.


### Training Scripts

Download the <link>[training scripts and data for this lab](https://drive.google.com/file/d/1ihaFWQLTsFPpzAfAHfhrt1cR4WEwhH7r/view)</link>

Move the code and data into your project directory and take a look at train.py and the code it uses. The code is originally from torchvision but modified by Dustin Franklin at NVIDIA for some smaller models that will run on the Jetson Nano, especially FCN-ResNet-18.

### Train on Pascal VOC

Use train.py to learn an initial model from the Pascal VOC 2012 (21 class) dataset. You'll want to train for about 100 epochs and take the model with the best IoU on the validation set. Expect about 52% IoU.

If you configured the runtime environment as above, the model weights after each iteration as well as the best model will be saved to /workspace in the container, which is mapped to $HOME/workspace on the host (puffer in our case).

Transfer learning: Fine tune on our robot floor dataset
Take a look at <code>retrain.py</code>. This script loads the pre-trained FCN-ResNet-18 we built in the previous step and fine tunes it on the robot floor data set. Take a look at the code and play around with it, only fine tuning the fresh output layer or also tuning the layers already trained on VOC. You'll also want to experiment with the relative weighting of floor vs. obstacle classes in the loss function. If you don't give obstacles (the rare class) a high weight, the model will learn to classify everything as floor!

I didn't put the code to save the model weights here. To get your model saved, read and understand the two scripts train.py and fine_tune.py and add the code to save your model to the fine tuning script.

Give some thought to what criteria you should use for the "best model so far." If you only cared about per-pixel accuracy, you would probably just classify all pixels as "floor." What you probably want to maximize is the mean of the IoU for the two classes.

### Export to ONNX and run under DNN

Once that's all working, export the model to ONNX using the provided script and see if you can get it running on OpenCV DNN.

### Put it all together!

Finally, you should be able to display, frame by frame, the input image with floor/non-floor pixels identified and a birds-eye view map of the obstacle-free space around
the robot (using the homography you saved earlier).

Post a video of your result to Piazza before the next lab, and turn in a report describing your experiments and results.
